In [ ]:
import torch
torch.__version__


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Author Oliver
# Date 2023/4/05 18:54
# load dataset

import pandas as pd
from pandas import read_excel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as skm
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib import pyplot

#load dataset

trainDataset = pd.read_csv('./processed_train.csv')
testDataset = pd.read_csv('./processed_test.csv')

print('\n=======================\n',trainDataset.shape)
print('\n=======================\n',testDataset.shape)

X_train = trainDataset.iloc[:,0:43]
Y_train = trainDataset.iloc[:,-1]
X_test = testDataset.iloc[:,0:43]

print(X_train.head())
print(Y_train.head())
print(X_test.head())

'''
#Normalization using normal scaler, result is having each feature in mean=0 and std=1
#print the test input features subset before normalization
print(X_test)
print(X_test.shape)
'''
#Normalization, both test and train features
scaler=StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
'''
#print the test input features, after normalization
print("\n------------------\n",X_test)
print(X_test.shape)
'''
import torch.nn as nn
import torch
from torch.utils.data import TensorDataset, DataLoader

#X_train.shape

X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()
X_test_tensor = torch.tensor(X_test).float()

train_data = TensorDataset(X_train,Y_train)
data = DataLoader(train_data,batch_size=32,shuffle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

#CNN
#hidden_size = [20,64,128,256,512]

class conv_model(nn.Module):
    def __init__(self):
        super().__init__()
        #self.hidden_size = hidden_size
        self.blocks = nn.ModuleList([])
        '''
        for i,size in enumerate(hidden_size):
            if i == 0:
                self.blocks.extend(
                    nn.ModuleList([
                        nn.Conv1d(1,hidden_size[i],kernel_size=2),nn.ReLU(hidden_size[i])
                    ])
                )
            else:
                self.blocks.extend(
                    nn.ModuleList([
                        nn.Conv1d(hidden_size[i-1],hidden_size[i],kernel_size=2),nn.ReLU(hidden_size[i])
                    ])
                )
        self.last_layer = nn.Linear(48,1)
        '''
        self.conv1 = nn.Conv1d(in_channels=43, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(1280, 256)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(256, 1)

        self.dropout = nn.Dropout(p=0.2)
    def forward(self,x):
        '''
        X = X.unsqueeze(1)
        for net in self.blocks:
            X = net(X)
        X = X.transpose(1,2)
        X = self.dropout(X)
        X = torch.flatten(X,start_dim=1)
        X = self.last_layer(X)
        X = X.squeeze()
        '''
        #print(x.shape)
        x = x.unsqueeze(1).expand(-1, 43, -1)
        #print(x.shape)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        x = x.view(-1, 1280)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = x.squeeze(-1)
        return x

trainMSEList = []
trainMAEList = []
#testMSEList = []
#testMAEList = []
epochList = []

epochs = 500
#model = conv_model(hidden_size = hidden_size)
model = conv_model()
model = model.float()
model = model.to(device)
#MSE
#Loss = nn.MSELoss()
#MAE
Loss1 = nn.L1Loss()
Optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)

for epoch in tqdm(range(epochs)):
    #batch_loss = []
    batch_loss1 = []
    for i,datal in enumerate(data):
        X,Y = datal
        X = X.to(device)
        Y = Y.to(device)
        Optimizer.zero_grad()
        output = model(X)
        #loss = Loss(output,Y)
        loss1 = Loss1(output,Y)

        #batch_loss.append(loss.detach().numpy())
        batch_loss1.append(loss1.detach().cpu().numpy())

        #loss.backward()
        loss1.backward()
        Optimizer.step()

    if epoch % 10==0:
        epochList.append(epoch)
        #trainMSEList.append(np.mean(batch_loss))
        trainMAEList.append(np.mean(batch_loss1))
        #print(f"epochs:{epoch},loss:{loss}")
        print(f"epochs:{epoch},loss:{loss1}")
        #Y_out = model(X_test)

        #testLoss=Loss(Y_out,Y_test)
        #testLoss1=Loss1(Y_out,Y_test)
        #testMSEList.append(testLoss.detach().numpy())
        #testMAEList.append(testLoss1.detach().numpy())

        #train_error = np.abs(Y_test.numpy()-Y_out.detach().numpy())
        #print(f"mean error:{np.mean(train_error)}\nmin error;{np.min(train_error)}\nmax error:{np.max(train_error)}")
        #print(f"mse:{loss.detach().numpy()}")
        print(f"mae:{loss1.detach().cpu().numpy()}")

        # define the file path to save the model
        PATH = "./result/model_dcnn_mae_1.pt"
        # save the model
        torch.save(model.state_dict(), PATH)


X_test_tensor = X_test_tensor.to(device)
Y_out = model(X_test_tensor)

# 将新列添加到数据集的最后一列
Y_out_np = Y_out.detach().cpu().numpy()
#X_test_dataframe = pd.DataFrame(X_test.detach().numpy())

testDataset_new = pd.read_csv('./processed_test.csv')
testDataset_new['resale_price'] = Y_out_np

# 保存新数据集
testDataset_new.to_csv('./result/resultAll_dcnn_mae.csv', index=False)

result = pd.DataFrame()
result.insert(result.shape[0], 'Id', range(len(Y_out_np)))
result.insert(1, 'Predicted', Y_out_np)
result.to_csv('./result/result_random_forest.csv', index = 0)

In [ ]:
#pyplot.plot(epochList,trainMSEList,color='blue',marker='o',linewidth=1,markersize=2,label='CNN_Train_MSE')
pyplot.plot(epochList,trainMAEList,color='red',marker='o',linewidth=1,markersize=2,label='CNN_Train_MAE')
pyplot.legend(loc=0)
#pyplot.title('CNN Train MSE')
pyplot.title('CNN Train MAE')
pyplot.xlabel("epoch")
#pyplot.ylabel("CNN Train MSE")
pyplot.title('CNN Train MAE')

In [14]:
import torch
import torch.nn as nn
import torchviz
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        #self.hidden_size = hidden_size
        self.blocks = nn.ModuleList([])
        self.conv1 = nn.Conv1d(in_channels=43, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(1280, 256)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self,x):
        #x = x.unsqueeze(1).expand(-1, 43, -1)
        #print(x.shape)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        x = x.view(-1, 1280)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = x.squeeze(-1)
        return x

model = Net()
inputs = torch.randn((1, 43, 1000))

dot = torchviz.make_dot(model(inputs), params=dict(model.named_parameters()))

# set the rankdir attribute to "LR" for left-to-right layout
dot.attr(rankdir='LR')

dot.render(filename='model_architecture', format='png')

'model_architecture.png'